## Testes da API

Nesse notebook são feitos alguns testes com a API do Olho Vivo

In [1]:
import pandas as pd
import time
import random
from requests import Session
from typing import Optional

In [2]:
class RequestMaker:

    def __init__(self, method:str, host:str, version:Optional[str]=None)->None:

        self.method = method
        self.host = host
        self.version = version
        self.session = Session()

        self.url_base = self.build_base_url()

    def build_base_url(self):

        url = f'{self.method}://{self.host}'

        if self.version:
            url = url+f'/v{self.version}'

        return url

    def build_params_str(self, params:dict)->str:

        params = [f"{key}={value}" for key, value in params.items()]
        params = '&'.join(params)
        params = f'?{params}'

        return params

    def build_request_url(self, endpoint:Optional[str]=None, params:Optional[dict]=None)->str:

        url = self.url_base
        if endpoint:
            url = self.url_base + '/' + endpoint.strip('/')
        
        if params:
            params_str = self.build_params_str(params)
            url = url + params_str
        
        return url
               
    def post(self,endpoint:Optional[str]=None, params:Optional[dict]=None, data:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo post em: {url}. Dados: {data}')
        with self.session.post(url, data=data) as r:
            if as_json:
                return r.json()
            return r.text
        
    def get(self, endpoint:Optional[str]=None, params:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo get em {url}.')
        with self.session.get(url) as r:
            if as_json:
                return r.json()
            return r.text



class APIOlhoVivoClient:

    HOST='api.olhovivo.sptrans.com.br'
    METHOD='http'
    VERSION='2.1'

    def __init__(self, token:str):

        self.request = RequestMaker(self.METHOD, self.HOST, self.VERSION)
        self.token=token

        self.autenticar()

    def autenticar(self)->None:

        endpoint = 'Login/Autenticar'
        params = {'token' : self.token}

        #a api nao retorna um json valido
        resp = self.request.post(endpoint, params, as_json=False)
        success = resp=='true'
        if not success:
            raise RuntimeError(f'Falha ao atenticar: {resp}')
        print('Autenticado com sucesso.')

    @property
    def posicao_atual_onibus(self):

        endpoint = 'Posicao'
        return self.request.get(endpoint)

In [3]:
token = '7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0'

In [4]:
api = APIOlhoVivoClient(token)

Fazendo post em: http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0. Dados: None
Autenticado com sucesso.


In [5]:
posicoes = []

#espera = 30
for i in range(1):
    posicao_atual = api.posicao_atual_onibus
    posicoes.append(posicao_atual)
    #time.sleep(espera)

posicoes

Fazendo get em http://api.olhovivo.sptrans.com.br/v2.1/Posicao.


[{'hr': '11:51',
  'l': [{'c': '917H-10',
    'cl': 456,
    'sl': 1,
    'lt0': 'METRÔ VL. MARIANA',
    'lt1': 'TERM. PIRITUBA',
    'qv': 13,
    'vs': [{'p': 11897,
      'a': True,
      'ta': '2025-09-16T14:50:49Z',
      'py': -23.5353285,
      'px': -46.712781750000005,
      'sv': None,
      'is': None},
     {'p': 11854,
      'a': True,
      'ta': '2025-09-16T14:51:13Z',
      'py': -23.515711874999997,
      'px': -46.753727,
      'sv': None,
      'is': None},
     {'p': 11904,
      'a': True,
      'ta': '2025-09-16T14:51:13Z',
      'py': -23.5000095,
      'px': -46.7471905,
      'sv': None,
      'is': None},
     {'p': 11471,
      'a': True,
      'ta': '2025-09-16T14:51:26Z',
      'py': -23.4865925,
      'px': -46.726296500000004,
      'sv': None,
      'is': None},
     {'p': 11909,
      'a': True,
      'ta': '2025-09-16T14:50:52Z',
      'py': -23.486014,
      'px': -46.7257505,
      'sv': None,
      'is': None},
     {'p': 11901,
      'a': True,
  

In [6]:
len(posicoes)

1

In [7]:
rows = []

for h in posicoes:
    if h:
        for l in h['l']:

            for veiculo in l['vs']:

                row = {
                    'linha': l['c'],
                    'id_onibus': veiculo['p'],
                    'timestamp': veiculo['ta'],
                    'lat': veiculo['py'],
                    'lon': veiculo['px'] 
                }

                rows.append(row)

df_posicoes = pd.DataFrame(rows)

Conversão de horário

In [8]:
df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,917H-10,11897,2025-09-16T14:50:49Z,-23.535328,-46.712782
1,917H-10,11854,2025-09-16T14:51:13Z,-23.515712,-46.753727
2,917H-10,11904,2025-09-16T14:51:13Z,-23.500010,-46.747191
3,917H-10,11471,2025-09-16T14:51:26Z,-23.486593,-46.726297
4,917H-10,11909,2025-09-16T14:50:52Z,-23.486014,-46.725750
...,...,...,...,...,...
10121,4004-10,48480,2025-09-16T14:51:10Z,-23.580201,-46.399039
10122,4086-10,47784,2025-09-16T14:50:50Z,-23.587460,-46.409183
10123,4008-21,48384,2025-09-16T14:51:25Z,-23.578815,-46.432018
10124,4016-10,48470,2025-09-16T14:51:20Z,-23.615679,-46.476302


In [9]:
df_posicoes['timestamp'] = pd.to_datetime(df_posicoes['timestamp'])

df_posicoes['timestamp'] = df_posicoes['timestamp'].dt.tz_convert("America/Sao_Paulo")

df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,917H-10,11897,2025-09-16 11:50:49-03:00,-23.535328,-46.712782
1,917H-10,11854,2025-09-16 11:51:13-03:00,-23.515712,-46.753727
2,917H-10,11904,2025-09-16 11:51:13-03:00,-23.500010,-46.747191
3,917H-10,11471,2025-09-16 11:51:26-03:00,-23.486593,-46.726297
4,917H-10,11909,2025-09-16 11:50:52-03:00,-23.486014,-46.725750
...,...,...,...,...,...
10121,4004-10,48480,2025-09-16 11:51:10-03:00,-23.580201,-46.399039
10122,4086-10,47784,2025-09-16 11:50:50-03:00,-23.587460,-46.409183
10123,4008-21,48384,2025-09-16 11:51:25-03:00,-23.578815,-46.432018
10124,4016-10,48470,2025-09-16 11:51:20-03:00,-23.615679,-46.476302


In [13]:
horas = df_posicoes["timestamp"].dt.hour

horas_unicas = horas.unique()
horas_unicas.sort()

print(horas_unicas)

[11]


Ônibus duplicados

In [11]:
duplicados = (df_posicoes.groupby(["id_onibus", "timestamp"]).filter(lambda x: len(x) > 1).sort_values(["id_onibus", "timestamp"]))

duplicados

,linha,id_onibus,timestamp,lat,lon
390,1178-10,31284,2025-09-16 11:51:07-03:00,-23.504361,-46.473071
8218,3902-10,31284,2025-09-16 11:51:07-03:00,-23.485104,-46.384289
